Minimum viable product is the ability to look at changes in drug pricing over time. More ideally, these data might be combined with population changes, stock changes, medicaid/medicare claims and/or enrollment data to search for correlations that improve prediction of future price.

In [193]:
# imports
import pandas as pd

In [194]:
# NDC drug names
# slow to read
fda_parent = pd.read_excel('/Users/albert/Documents/Capstone/Supplements/ndcxls/product.xlsx')

In [195]:
# Medicaid/medicare pricing data
pricing_parent = pd.read_csv('data/data.csv')

In [196]:
# I only need some of the columns available; they are also reordered here
fda = fda_parent[['PRODUCTNDC','NONPROPRIETARYNAME','SUBSTANCENAME','PROPRIETARYNAME','PHARM_CLASSES']]
fda.head()

,PRODUCTNDC,NONPROPRIETARYNAME,SUBSTANCENAME,PROPRIETARYNAME,PHARM_CLASSES
0,0002-0800,diluent,WATER,Sterile Diluent,NaN
1,0002-1200,Florbetapir F 18,FLORBETAPIR F-18,Amyvid,"Positron Emitting Activity [MoA], Radioactive ..."
2,0002-1210,Flortaucipir F-18,FLORTAUCIPIR F-18,TAUVID,NaN
3,0002-1220,Flortaucipir F-18,FLORTAUCIPIR F-18,TAUVID,NaN
4,0002-1433,Dulaglutide,DULAGLUTIDE,Trulicity,"GLP-1 Receptor Agonist [EPC], Glucagon-Like Pe..."


In [197]:
# importing medicare/medicaid data
pricing_parent['ndc'] = pricing_parent['ndc'].astype(str)  # ERROR I THINK LEFT PADDING ZEROS ARE LOST ON INITIAL IMPORT???
# only specific columns are needed
pricing = pricing_parent[['ndc','ndc_description','nadac_per_unit']]
pricing.head()

,ndc,ndc_description,nadac_per_unit
0,24385005452,12HR NASAL DECONGEST ER 120 MG,0.29229
1,70000047501,12HR NASAL DECONGEST ER 120 MG,0.29229
2,46122038576,24H NASAL ALLERGY 55 MCG SPRAY,0.72102
3,70000036201,24HR ALLERGY(LEVOCETIRZN) 5 MG,0.19182
4,70000036202,24HR ALLERGY(LEVOCETIRZN) 5 MG,0.19182


In [198]:
# remove hyphen/dash in NDC for fda
fda['PRODUCTNDC'] = fda['PRODUCTNDC'].str.replace('-', '')
fda.head()

/var/folders/zd/qjtyx19s513b8nwpd06dxsmr0000gn/T/ipykernel_67063/3870925609.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fda['PRODUCTNDC'] = fda['PRODUCTNDC'].str.replace('-', '')


,PRODUCTNDC,NONPROPRIETARYNAME,SUBSTANCENAME,PROPRIETARYNAME,PHARM_CLASSES
0,00020800,diluent,WATER,Sterile Diluent,NaN
1,00021200,Florbetapir F 18,FLORBETAPIR F-18,Amyvid,"Positron Emitting Activity [MoA], Radioactive ..."
2,00021210,Flortaucipir F-18,FLORTAUCIPIR F-18,TAUVID,NaN
3,00021220,Flortaucipir F-18,FLORTAUCIPIR F-18,TAUVID,NaN
4,00021433,Dulaglutide,DULAGLUTIDE,Trulicity,"GLP-1 Receptor Agonist [EPC], Glucagon-Like Pe..."


In [199]:
fda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104293 entries, 0 to 104292
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   PRODUCTNDC          104293 non-null  object
 1   NONPROPRIETARYNAME  104289 non-null  object
 2   SUBSTANCENAME       102001 non-null  object
 3   PROPRIETARYNAME     104274 non-null  object
 4   PHARM_CLASSES       64955 non-null   object
dtypes: object(5)
memory usage: 4.0+ MB


In [200]:
# the length of the NDCs are not the same making this more involved than it ought to be
print(fda['PRODUCTNDC'].str.len().mean())
print(pricing['ndc'].str.len().mean())

8.288801741248214
10.527122418934733


In [201]:
# trying to get the number of terms present common to both ndc lists
fda['PRODUCTNDC'].isin(pricing['ndc']).sum()
len(set(fda['PRODUCTNDC'])&set(pricing['ndc']))

0

okay this is messy let's reduce to a smaller set to work with like maybe only zinc supplements

In [202]:
fda.loc[fda['NONPROPRIETARYNAME']=='Zinc']

,PRODUCTNDC,NONPROPRIETARYNAME,SUBSTANCENAME,PROPRIETARYNAME,PHARM_CLASSES
57356,621065367,Zinc,ZINC,ZINCUM METALLICUM,NaN


There are too many and too few zinc so I'll use 'abacavir' since I know that there are common NDCs between the two files

ran into an issue where there are null values in the fda list, so I can't search by names. Need to remove nulls; there are only 4

In [203]:
no_names = fda[fda['NONPROPRIETARYNAME'].isnull()].index
no_names = no_names.to_list()
print(f"There are {len(no_names)} rows without nonproprietary names. The rows are {no_names} and the substances are {list(fda['SUBSTANCENAME'].loc[no_names])}.")

There are 4 rows without nonproprietary names. The rows are [38715, 38718, 51039, 99618] and the substances are ['BENZALKONIUM CHLORIDE', 'BENZALKONIUM CHLORIDE', 'AVOBENZONE; OCTOCRYLENE; OXYBENZONE', 'BENZALKONIUM CHLORIDE'].


In [204]:
fda.drop(index=no_names,inplace=True)
no_names = fda[fda['NONPROPRIETARYNAME'].isnull()].index
no_names = no_names.to_list()
print(f"There are {len(no_names)} rows without nonproprietary names. The rows are {no_names} and the substances are {list(fda['SUBSTANCENAME'].loc[no_names])}.")

There are 0 rows without nonproprietary names. The rows are [] and the substances are [].


/var/folders/zd/qjtyx19s513b8nwpd06dxsmr0000gn/T/ipykernel_67063/1314845254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fda.drop(index=no_names,inplace=True)


In [205]:
fda.drop(index=no_names,inplace=True)
no_names = fda[fda['SUBSTANCENAME'].isnull()].index
no_names = no_names.to_list()
print(no_names)
# print(f"There are {len(no_names)} rows without nonproprietary names. The rows are {no_names} and the substances are {list(fda['SUBSTANCENAME'].loc[no_names])}.")

[67, 68, 69, 84, 85, 86, 87, 142, 163, 209, 241, 253, 280, 385, 386, 387, 388, 389, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 472, 482, 520, 521, 526, 529, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 677, 734, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 1032, 1033, 1040, 1075, 1079, 1081, 1088, 1094, 1101, 1103, 1104, 1105, 1109, 1153, 1162, 1168, 1186, 1187, 1317, 1318, 1319, 1321, 1325, 1326, 1327, 1328, 1332, 1333, 1334, 1338, 1339, 1343, 1344, 1350, 1359, 1361, 1362, 1363, 1364, 1365, 1367, 1368, 1371, 1406, 1508, 1548, 1562, 1564, 1565, 1578, 1580, 1581, 1598, 1620, 1621, 1622, 1721, 1767, 1784, 1802, 1814, 1867, 1880, 1913, 1914, 2002, 2017, 2086, 2091, 2092, 2155, 2248, 2285, 2294, 2296, 2321, 2324, 2327, 2331, 2343, 2348, 2350, 2711, 2849, 3299, 3300, 3301, 3302, 3303, 33

/var/folders/zd/qjtyx19s513b8nwpd06dxsmr0000gn/T/ipykernel_67063/4057272107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fda.drop(index=no_names,inplace=True)


In [206]:
# so that I don't have to keep making changes just putting drug in a variable called var
var = 'abacavir'

In [207]:
fda['NONPROPRIETARYNAME'] = fda['NONPROPRIETARYNAME'].str.lower()
var_fda = fda[fda['NONPROPRIETARYNAME'].str.contains(var)]
print(len(var_fda))
# print(var_fda)

# var_fda = fda[fda['NONPROPRIETARYNAME'].str.contains(var)] 
# print(len(var_fda))
# # print(var_fda) 

27


/var/folders/zd/qjtyx19s513b8nwpd06dxsmr0000gn/T/ipykernel_67063/969232306.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fda['NONPROPRIETARYNAME'] = fda['NONPROPRIETARYNAME'].str.lower()


In [208]:
pricing['ndc_description'] = pricing['ndc_description'].str.lower()
var_pricing = pricing[pricing['ndc_description'].str.contains(var)]
print(len(var_pricing))

18


/var/folders/zd/qjtyx19s513b8nwpd06dxsmr0000gn/T/ipykernel_67063/1859290897.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pricing['ndc_description'] = pricing['ndc_description'].str.lower()


In [211]:
similar = pd.concat([var_pricing['ndc'],var_fda['PRODUCTNDC']], axis=1)
similar

,ndc,PRODUCTNDC
12,378410591,NaN
13,904687404,NaN
14,904687406,NaN
15,31722055760,NaN
16,50268004911,NaN
17,50268004912,NaN
18,51079020401,NaN
19,51079020406,NaN
20,64380071703,NaN
21,65862007360,NaN


FDA data do not include the last 2 digits relating to packaging/dosage for NDC.

Documentation says that medicaid/medicare data DOES include the last two digits; but this does not look to be the case. Documentation also says **incorrectly** that NDC is 11 digits.